# From Keras to Edge TPU

To make a TF model compatible with Coral Edge TPU, a TFLite __full integer quantization__ is needed. Read _point 4_ of the previous section for how it works.

After the TF Lite conversion, the __Edge TPU Compiler__ has to be used to get the final model. The output of this final conversion is still a tflite model, but can be deployed on the Coral Dev Board or the Coral USB Accelerator.

In [1]:
import tensorflow as tf
import numpy as np
import os
import pathlib
import subprocess

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
print(gpus[0])

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [3]:
# set the folder path where is located the model 
DIR = './models'
model_name = 'model'

model_fp = os.path.join(DIR, model_name) + '.h5'

In [4]:
model = tf.keras.models.load_model(model_fp)
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 100, 100, 16)      448       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 50, 50, 16)        2320      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 25, 25, 16)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 25, 25, 8)         1160      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 12, 12, 8)         0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 1152)            

## Full integer quantization

In [5]:
input_size = (100,100)  # input size of the model
data_range = 255  # range to normalize data
n_data = 100      # number of representative inputs

DATASET_DIR = './dataset' #the directory must contain at least n_data images

dataset_fp = pathlib.Path(DATASET_DIR)

### Import the dataset

In [6]:
list_ds = tf.data.Dataset.list_files(str(dataset_fp/"*"))

def representative_dataset_gen():
    for _ in range(n_data):
        for img_f in list_ds.take(1):
            img = tf.io.decode_image(tf.io.read_file(img_f), channels=3, dtype=tf.dtypes.uint8)
            img = tf.image.resize(img, input_size, method=tf.image.ResizeMethod.AREA)
            yield ([img[None]])

### Create the converter object

In [7]:
# import the converter loading the model
converter = tf.compat.v1.lite.TFLiteConverter.from_keras_model_file(model_fp) #TF2.0 currently not compatible

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

### Convert the model and save it

In [8]:
tflite_full_integer_model = converter.convert()

tflite_model_full_integer_file = os.path.join(DIR,model_name) + "_full_integer.tflite"
pathlib.Path(tflite_model_full_integer_file).write_bytes(tflite_full_integer_model)
print(f"Model {tflite_model_full_integer_file} saved.")

Model ./models/model_full_integer.tflite saved.


## Edge TPU compilation

Warning: __edgetpu_compiler__ has to be installed on the system!

In [9]:
out = subprocess.check_output(f"edgetpu_compiler {tflite_model_full_integer_file} -s -m 13 -o ./models",shell=True).decode("utf-8")
print(out)

Edge TPU Compiler version 2.0.291256449

Model compiled successfully in 29 ms.

Input model: ./models/model_full_integer.tflite
Input size: 81.44KiB
Output model: ./models/model_full_integer_edgetpu.tflite
Output size: 144.54KiB
On-chip memory available for caching model parameters: 7.29MiB
On-chip memory used for caching model parameters: 99.00KiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 11
Operation log: ./models/model_full_integer_edgetpu.log

Operator                       Count      Status

SOFTMAX                        1          Mapped to Edge TPU
FULLY_CONNECTED                2          Mapped to Edge TPU
MAX_POOL_2D                    3          Mapped to Edge TPU
QUANTIZE                       2          Mapped to Edge TPU
CONV_2D                        3          Mapped to Edge TPU

